In [17]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [18]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4refnu1w
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4refnu1w
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [19]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


**Kernal1  (Each thread produces one output matrix element)**

In [20]:

%%writefile Norhan_Reda_Abdelwahed_2_31_kernal1.cu
//Norhan_Reda_Abdelwahed_2_31
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
using namespace std;
#define N 800
// Kernel definition
__global__ void MatAdd(float A[N][N], float B[N][N],
                       float C[N][N])
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;
    if (i < N && j < N)
        C[i][j] = A[i][j] + B[i][j];


}

int main()
{
      float A[N][N];
    float B[N][N];
    float C[N][N];

    // Initialize matrices A and B
    for (int i = 0; i < N; i++)
    {
        for (int j = 0; j < N; j++)
        {
            A[i][j] = 1.0f; // Example initialization
            B[i][j] = 2.0f; // Example initialization
        }
    }


    // Declare device matrices
    float (*d_A)[N];
    float (*d_B)[N];
    float (*d_C)[N];

    // Allocate device memory for matrices A, B, and C
    cudaMalloc((void**)&d_A, N * N * sizeof(float));
    cudaMalloc((void**)&d_B, N * N * sizeof(float));
    cudaMalloc((void**)&d_C, N * N * sizeof(float));

    // Copy input matrices A and B from host to device
    cudaMemcpy(d_A, A, N * N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, N * N * sizeof(float), cudaMemcpyHostToDevice);



    //...
    // Kernel invocation with one block of N * N * 1 threads
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks(N / threadsPerBlock.x, N / threadsPerBlock.y);
    MatAdd<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C);
    //...
    // Copy the result matrix C from device to host
    cudaMemcpy(C, d_C, N * N * sizeof(float), cudaMemcpyDeviceToHost);
   for(int i=0;i<5;i++)
   {
    for(int j = 0;j<5;j++)
    {
      cout<<C[i][j]<<endl;
    }
   }
    printf("C[0][0] = %f\n", C[0][0]);
    printf("PASSED\n");

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

   return 0;
}


Overwriting Norhan_Reda_Abdelwahed_2_31_kernal1.cu


In [21]:
!chmod +x Norhan_Reda_Abdelwahed_2_31_kernal1.cu
!nvcc Norhan_Reda_Abdelwahed_2_31_kernal1.cu -o kernal1
!./kernal1

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
C[0][0] = 3.000000
PASSED


In [22]:
!nvprof ./kernal1

==4768== NVPROF is profiling process 4768, command: ./kernal1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
C[0][0] = 3.000000
PASSED
==4768== Profiling application: ./kernal1
==4768== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   62.36%  809.42us         2  404.71us  397.78us  411.64us  [CUDA memcpy HtoD]
                   30.38%  394.39us         1  394.39us  394.39us  394.39us  [CUDA memcpy DtoH]
                    7.26%  94.174us         1  94.174us  94.174us  94.174us  MatAdd(float[800]*, float[800]*, float[800]*)
      API calls:   91.65%  101.94ms         3  33.981ms  118.95us  101.70ms  cudaMalloc
                    7.07%  7.8679ms         3  2.6226ms  641.98us  6.5589ms  cudaMemcpy
                    0.88%  981.02us         3  327.01us  170.22us  408.04us  cudaFree
                    0.24%  263.35us         1  263.35us  263.35us  263.35us  cudaLaunchKernel
                    0.13%  145.46us     


**Kernal 2 ( Each thread produces one output matrix row)**

In [23]:

%%writefile Norhan_Reda_Abdelwahed_2_31_kernal2.cu
//Norhan_Reda_Abdelwahed_2_31
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
using namespace std;
#define N 800
// Kernel definition
__global__ void MatAdd(float A[N][N], float B[N][N],
                       float C[N][N])
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
     for (int j = 0; j < N; j++) {
        if (i < N)
            C[i][j] = A[i][j] + B[i][j];
    }


}

int main()
{
      float A[N][N];
    float B[N][N];
    float C[N][N];

    // Initialize matrices A and B
    for (int i = 0; i < N; i++)
    {
        for (int j = 0; j < N; j++)
        {
            A[i][j] = 1.0f; // Example initialization
            B[i][j] = 2.0f; // Example initialization
        }
    }


    // Declare device matrices
    float (*d_A)[N];
    float (*d_B)[N];
    float (*d_C)[N];

    // Allocate device memory for matrices A, B, and C
    cudaMalloc((void**)&d_A, N * N * sizeof(float));
    cudaMalloc((void**)&d_B, N * N * sizeof(float));
    cudaMalloc((void**)&d_C, N * N * sizeof(float));

    // Copy input matrices A and B from host to device
    cudaMemcpy(d_A, A, N * N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, N * N * sizeof(float), cudaMemcpyHostToDevice);



    //...
    // Kernel invocation with one block of N * N * 1 threads
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks(N / threadsPerBlock.x, N / threadsPerBlock.y);
    MatAdd<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C);
    //...
    // Copy the result matrix C from device to host
    cudaMemcpy(C, d_C, N * N * sizeof(float), cudaMemcpyDeviceToHost);
   for(int i=0;i<5;i++)
   {
    for(int j = 0;j<5;j++)
    {
      cout<<C[i][j]<<endl;
    }
   }
    printf("C[0][0] = %f\n", C[0][0]);
    printf("PASSED\n");

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

   return 0;
}


Writing Norhan_Reda_Abdelwahed_2_31_kernal2.cu


In [24]:
!chmod +x Norhan_Reda_Abdelwahed_2_31_kernal2.cu
!nvcc Norhan_Reda_Abdelwahed_2_31_kernal2.cu -o kernal2
!./kernal2

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
C[0][0] = 3.000000
PASSED


In [25]:
!nvprof ./kernal2

==4841== NVPROF is profiling process 4841, command: ./kernal2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
C[0][0] = 3.000000
PASSED
==4841== Profiling application: ./kernal2
==4841== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   98.18%  66.491ms         1  66.491ms  66.491ms  66.491ms  MatAdd(float[800]*, float[800]*, float[800]*)
                    1.16%  785.10us         2  392.55us  377.62us  407.48us  [CUDA memcpy HtoD]
                    0.66%  446.68us         1  446.68us  446.68us  446.68us  [CUDA memcpy DtoH]
      API calls:   59.46%  102.47ms         3  34.156ms  125.68us  102.21ms  cudaMalloc
                   39.84%  68.656ms         3  22.885ms  607.12us  67.383ms  cudaMemcpy
                    0.42%  717.06us         3  239.02us  224.64us  257.83us  cudaFree
                    0.15%  255.14us         1  255.14us  255.14us  255.14us  cudaLaunchKernel
                    0.12%  201.03us     

**Kernal 3 (Each thread produces one output matrix column)**

In [26]:

%%writefile Norhan_Reda_Abdelwahed_2_31_kernal3.cu
//Norhan_Reda_Abdelwahed_2_31
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
using namespace std;
#define N 800
// Kernel definition
__global__ void MatAdd(float A[N][N], float B[N][N],
                       float C[N][N])
{

    int j = blockIdx.y * blockDim.y + threadIdx.y;
    for (int i = 0; i < N; i++) {
        if (j < N)
            C[i][j] = A[i][j] + B[i][j];
    }


}

int main()
{
      float A[N][N];
    float B[N][N];
    float C[N][N];

    // Initialize matrices A and B
    for (int i = 0; i < N; i++)
    {
        for (int j = 0; j < N; j++)
        {
            A[i][j] = 1.0f; // Example initialization
            B[i][j] = 2.0f; // Example initialization
        }
    }


    // Declare device matrices
    float (*d_A)[N];
    float (*d_B)[N];
    float (*d_C)[N];

    // Allocate device memory for matrices A, B, and C
    cudaMalloc((void**)&d_A, N * N * sizeof(float));
    cudaMalloc((void**)&d_B, N * N * sizeof(float));
    cudaMalloc((void**)&d_C, N * N * sizeof(float));

    // Copy input matrices A and B from host to device
    cudaMemcpy(d_A, A, N * N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, N * N * sizeof(float), cudaMemcpyHostToDevice);



    //...
    // Kernel invocation with one block of N * N * 1 threads
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks(N / threadsPerBlock.x, N / threadsPerBlock.y);
    MatAdd<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C);
    //...
    // Copy the result matrix C from device to host
    cudaMemcpy(C, d_C, N * N * sizeof(float), cudaMemcpyDeviceToHost);
   for(int i=0;i<5;i++)
   {
    for(int j = 0;j<5;j++)
    {
      cout<<C[i][j]<<endl;
    }
   }
    printf("C[0][0] = %f\n", C[0][0]);
    printf("PASSED\n");

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

   return 0;
}


Writing Norhan_Reda_Abdelwahed_2_31_kernal3.cu


In [27]:
!chmod +x Norhan_Reda_Abdelwahed_2_31_kernal3.cu
!nvcc Norhan_Reda_Abdelwahed_2_31_kernal3.cu -o kernal3
!./kernal3

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
C[0][0] = 3.000000
PASSED


In [28]:
!nvprof ./kernal3

==4901== NVPROF is profiling process 4901, command: ./kernal3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
C[0][0] = 3.000000
PASSED
==4901== Profiling application: ./kernal3
==4901== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   84.17%  6.1696ms         1  6.1696ms  6.1696ms  6.1696ms  MatAdd(float[800]*, float[800]*, float[800]*)
                   10.57%  774.45us         2  387.22us  360.89us  413.56us  [CUDA memcpy HtoD]
                    5.26%  385.66us         1  385.66us  385.66us  385.66us  [CUDA memcpy DtoH]
      API calls:   89.19%  74.456ms         3  24.819ms  77.818us  74.298ms  cudaMalloc
                    9.68%  8.0778ms         3  2.6926ms  579.25us  6.8745ms  cudaMemcpy
                    0.65%  545.92us         3  181.97us  129.77us  208.34us  cudaFree
                    0.28%  232.79us         1  232.79us  232.79us  232.79us  cudaLaunchKernel
                    0.16%  136.24us     